In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic

In [ ]:
#QPU connection
try:
    from qat.qlmaas import QLMaaSConnection
    connection = QLMaaSConnection()
    LinAlg = connection.get_qpu("qat.qpus:LinAlg")
    lineal_qpu = LinAlg()
except ImportError:
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()

In [ ]:
sys.path.append('/home/cesga/gferro/NEASQC/PhaseAmplification/')

In [ ]:
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
from dataloading_module import LeftConditionalProbability, CRBS_gate
from qat.core.console import display
from qat.lang.AQASM import Program, RY, AbstractGate, QRoutine, X

In [ ]:
from AuxiliarFunctions import TestBins, LeftConditionalProbability, get_histogram, PostProcessResults

In [ ]:
#number of Qbits for the circuit
n_qbits = 8
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

x_X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(x_X)
ProbDict = {'array': p_X}

#  BARRERA

In [ ]:
def CRBS_generator(N, ControlState, Theta):
    """
    This functions codify a input ControlState using N qbits and
    apply a controlled Rotation of an input angle Theta by the ControlState
    on one aditional qbit.
    Inputs:
    * N: int. Number of qbits needed for codify the ControlState. 
    * ControlState: int. State for controlling the of the controlled Rotation.
    * Theta: float. Rotation angle    
    """
    qrout = QRoutine()
    
    #Creates de control using first N
    qcontrol = qrout.new_wires(N)
    #An additional target qbit  
    qtarget = qrout.new_wires(1)    
    
    #Transform staje in binnary string
    bNumber = list(format(ControlState, '0{}b'.format(int(N))))
    #Binnary string to list of Booleans
    bList = [bool(int(i)) for i in bNumber]
    
    #This block contains the mandatory transformation to use the ControlState 
    #for performing a controlled Operation on the target qbit
    for m, k in enumerate(bList):
        if k == False:
            qrout.apply(X, qcontrol[m])
            
    #Apply the controlled rotation on the target qbit
    #The rotation is only applyied when qcontrol is in ControlState
    c_i_RY = RY(Theta).ctrl(len(qcontrol))
    qrout.apply(c_i_RY, qcontrol, qtarget)
    #Undo the operations for using the ControlState
    #for controlling the rotation
    for m, k in enumerate(bList):
        if k == False:
            qrout.apply(X,qcontrol[m])           
    return qrout    

#Using generator function an abstract gate is created
CRBS_gate = AbstractGate("CRBS_Gate", [int, int, float])   
CRBS_gate.set_circuit_generator(CRBS_generator)



In [ ]:
def P_generator(n_qbits, Dictionary):
    
    ProbabilityArray = Dictionary['array']
    n_qbits = TestBins(ProbabilityArray, 'Probability')
    
    qrout = QRoutine()
    qbits = qrout.new_wires(n_qbits)
    nbins = len(ProbabilityArray)
    
    for i in range(0, n_qbits):
        ConditionalProbability = LeftConditionalProbability(i, ProbabilityArray)
        Thetas = 2.0*(np.arccos(np.sqrt(ConditionalProbability)))    

        if i == 0:
            #The first qbit is a typical y Rotation
            qrout.apply(RY(Thetas[0]), qbits[0])
        else:
            #The different rotations should be applied  over the i+1 qbit.
            #Each rotation is controlled by all the posible states formed with i qbits
            for j, theta in enumerate(Thetas):
                #Next lines do the following operation: |j> x Ry(2*\theta_{j})|0>
                gate = CRBS_gate(i, j, theta)
                qrout.apply(gate, qbits[:i+1])    
    return qrout


LoadP_Gate = AbstractGate(
    "P_Gate", 
    [int, dict],
    circuit_generator = P_generator,
    arity = lambda x,y:x
)




In [ ]:
P_gate = LoadP_Gate(n_qbits, ProbDict)

In [ ]:
%qatdisplay P_gate --depth 0

In [ ]:
qprog = Program()
qbits = qprog.qalloc(P_gate.arity)
qprog.apply(P_gate, qbits)

In [ ]:
circuit = qprog.to_circ()#link=[P_generator])

In [ ]:
%qatdisplay circuit --depth 0

In [ ]:
circuit.statistics()

In [ ]:
job = circuit.to_job()

In [ ]:
job = circuit.to_job()
result = lineal_qpu.submit(job)
P_results = PostProcessResults(result.join())

In [ ]:
from qat.

In [ ]:
def CreatePG(ProbabilityArray):
    """
    Given a discretized probability array the function creates a AbstracGate that allows the load
    of the probability in a Quantum State. The number of qbits of the gate will be log2(len(ProbabilityArray))
    Inputs:
    * ProbabilityArray: np.array. Discretized arrray with the probability to load
    Outuput:
    * P_gate: Abstract Gate. Gate for loading Input probability in a quantum state
    """
    
    #Number of Input qbits for the QWuantum Gate
    #nqbits_ = np.log2(len(ProbabilityArray))
    ##Probability array must have a dimension of 2^n.
    #Condition = (nqbits_%2 ==0) or (nqbits_%2 ==1)
    #if Condition == False:
    #    raise ValueError(
    #        'Length of the ProbabilityArray must be of dimension 2^n with n a int. In this case is: {}.'.format(
    #            nqbits_
    #        )
    #    )
    #nqbits_ = int(nqbits_)
    nqbits_ = TestBins(ProbabilityArray, 'Probability')
    def LoadProbability_generator(NumbeOfQbits):
        
        qrout = QRoutine()
        qbits = qrout.new_wires(NumbeOfQbits)
        nbins = 2**NumbeOfQbits        
        
        #Iteratively generation of the circuit
        for i in range(0, NumbeOfQbits):
            #Each step divides the bins in the step before by 2:
            #if i=1 -> there are 2 divisions so the step splits each one in 2 so 4 new bins are generated
            #if i=2 -> there are 4 divisions so the step split each one in 2 so 8 new bins are generated
            
            #Calculates Conditional Probability
            ConditionalProbability = LeftConditionalProbability(i, ProbabilityArray)
            #Rotation angles: length: 2^(i-1)-1 and i the number of qbits of the step
            Thetas = 2.0*(np.arccos(np.sqrt(ConditionalProbability)))

            if i == 0:
                #The first qbit is a typical y Rotation
                qrout.apply(RY(Thetas[0]), qbits[0])
            else:
                #The different rotations should be applied  over the i+1 qbit.
                #Each rotation is controlled by all the posible states formed with i qbits
                for j, theta in enumerate(Thetas):
                    #Next lines do the following operation: |j> x Ry(2*\theta_{j})|0>
                    gate = CRBS_gate(i, j, theta)
                    qrout.apply(gate, qbits[:i+1])    
        return qrout
    
    LoadP_Gate = AbstractGate("P_Gate", [int])   
    LoadP_Gate.set_circuit_generator(LoadProbability_generator)
    #Now We generated the complete Quantum Gate
    P_gate = LoadP_Gate(nqbits_)
    return P_gate   

p = AbstractGate("P_Gate", [int, dict], arity=lambda x, y: x)

In [ ]:
from qat.lang.AQASM.misc import build_gate
@build_gate("P_Gate", [int, dict], lambda x,y: x)
def P_generator(n_qbits, Dictionary):
    
    qrout = QRoutine()
    qbits = qrout.new_wires(n_qbits)
    nbins = 2**n_qbits  
    ProbabilityArray = Dictionary['array']    
    for i in range(0, n_qbits):
        ConditionalProbability = LeftConditionalProbability(i, ProbabilityArray)
        Thetas = 2.0*(np.arccos(np.sqrt(ConditionalProbability)))    
        if i == 0:
            qrout.apply(RY(Thetas[0]), qbits[0])
        else:
            for j, theta in enumerate(Thetas):
                bNumber = list(format(j, '0{}b'.format(i)))
                bList = [bool(int(i)) for i in bNumber]
                for m, k in enumerate(bList):
                    if k == False:
                        qrout.apply(X, qbits[m])
                c_i_RY = RY(theta).ctrl(i)
                qrout.apply(c_i_RY, qbits[:i], qbits[i])
                for m, k in enumerate(bList):
                    if k == False:
                        qrout.apply(X, qbits[m])
    return qrout

In [ ]:
P_Gate = p(n_qbits, ProbDict)

In [ ]:
P_Gate= P_generator(n_qbits, ProbDict)

In [ ]:
qprog = Program()

In [ ]:
qbits = qprog.qalloc(P_Gate.arity)

In [ ]:
qprog.apply(P_Gate, qbits)

In [ ]:
def LoadP(nqbits, p_X):
    ProbDict = {'array': p_X}
    qrout = QRoutine()
    qbits = qrout.new_wires(nqbits)
    with qrout.compute():
        qrout.apply(p(nqbits, ProbDict), qbits)
    return qrout
    
    
    
    

In [ ]:
circuit = LoadP(n_qbits, p_X)

In [ ]:
P_Gate= P_generator(n_qbits, ProbDict)

In [ ]:
%qatdisplay P_Gate --depth 1

In [ ]:
qprog = Program()
qbits = qprog.qalloc(n_qbits)
qprog.apply(P_Gate, qbits)

In [ ]:
circuit = qprog.to_circ()

In [ ]:
%qatdisplay circuit --depth 1

In [ ]:
np.isclose(P_results['Probability'], p_X).all()

In [ ]:
from qat.lang.AQASM.misc import build_gate

#@build_gate("P_Gate", [int, dict], lambda x,y: x)
def LoadProbability_generator(NumbeOfQbits, ProbabilityDict):

    qrout = QRoutine()
    qbits = qrout.new_wires(NumbeOfQbits)
    nbins = 2**NumbeOfQbits  
    ProbabilityArray = ProbabilityDict['array']
    
    for i in range(0, NumbeOfQbits):
        ConditionalProbability = LeftConditionalProbability(i, ProbabilityArray)
        Thetas = 2.0*(np.arccos(np.sqrt(ConditionalProbability)))
        
        if i == 0:
            qrout.apply(RY(Thetas[0]), qbits[0])
        else:
            for j, theta in enumerate(Thetas):
                bNumber = list(format(j, '0{}b'.format(i)))
                bList = [bool(int(i)) for i in bNumber]
                for m, k in enumerate(bList):
                    if k == False:
                        qrout.apply(X, qbits[m])
                c_i_RY = RY(theta).ctrl(i)
                qrout.apply(c_i_RY, qbits[:i], qbits[i])
                for m, k in enumerate(bList):
                    if k == False:
                        qrout.apply(X, qbits[m])
                #gate = CRBS_gate(i, j, theta)
                #qrout.apply(gate, qbits[:i+1]) 
    return qrout

LoadP_Gate = AbstractGate(
    'P_Gate',
    [int, dict],
    circuit_generator=LoadProbability_generator
)

In [ ]:
P_Gate= LoadP_Gate(n_qbits, ProbDict)

In [ ]:
qprog = Program()
qbits = qprog.qalloc(n_qbits)
qprog.apply(P_Gate, qbits)

In [ ]:
np.isclose(P_results['Probability'], p_X).all()